In [1]:
from datasets import load_dataset, concatenate_datasets

LOCAL_REPO = "CS2CD.Counter-Strike_2_Cheat_Detection"


C:\Users\kilua\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from glob import glob
import pandas as pd

primer_parquet = glob(f"{LOCAL_REPO}/with_cheater_present/*.parquet")[0]

df_ejemplo = pd.read_parquet(primer_parquet)
print("Columnas en el primer shard de no_cheater_present:")
print(df_ejemplo.columns.tolist())
df_ejemplo.head()

Columnas en el primer shard de no_cheater_present:
['inventory', 'usercmd_input_history', 'inventory_as_ids', 'approximate_spotted_by', 'aim_punch_angle_vel', 'aim_punch_angle', 'active_weapon_ammo', 'item_def_idx', 'weapon_quality', 'entity_lvl', 'item_id_high', 'item_id_low', 'inventory_position', 'is_initialized', 'econ_item_attribute_def_idx', 'initial_value', 'refundable_currency', 'set_bonus', 'fire_seq_start_time', 'fire_seq_start_time_change', 'm_iState', 'weapon_mode', 'accuracy_penalty', 'i_recoil_idx', 'fl_recoil_idx', 'is_burst_mode', 'is_in_reload', 'reload_visually_complete', 'dropped_at_time', 'is_hauled_back', 'is_silencer_on', 'time_silencer_switch_complete', 'orig_team_number', 'prev_owner', 'last_shot_time', 'iron_sight_mode', 'num_empty_attacks', 'zoom_lvl', 'burst_shots_remaining', 'needs_bolt_action', 'next_primary_attack_tick', 'next_primary_attack_tick_ratio', 'next_secondary_attack_tick', 'next_secondary_attack_tick_ratio', 'total_ammo_left', 'is_freeze_period'

,inventory,usercmd_input_history,inventory_as_ids,approximate_spotted_by,aim_punch_angle_vel,aim_punch_angle,active_weapon_ammo,item_def_idx,weapon_quality,entity_lvl,...,usercmd_left_move,usercmd_forward_move,usercmd_consumed_server_angle_changes,is_alive,velocity_Z,active_weapon_name,usercmd_mouse_dx,game_time,tick,steamid
0,"[knife_t, Glock-18]",[],"[59, 4]",[],"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]",0.0,59.0,0.0,1.0,...,NaN,NaN,NaN,True,NaN,knife_t,NaN,51.1875,1,Player_8
1,"[knife_t, Glock-18, AK-47]",[],"[59, 4, 7]",[],"[0.0, 0.0, 0.0]","[0.4380251467227936, 0.01679001934826374, 0.0]",30.0,7.0,4.0,1.0,...,NaN,NaN,NaN,True,NaN,AK-47,NaN,51.1875,1,Player_9
2,"[knife_t, Glock-18]",[],"[59, 4]",[],"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]",20.0,4.0,4.0,1.0,...,NaN,NaN,NaN,True,NaN,Glock-18,NaN,51.1875,1,Player_7
3,"[knife, USP-S]",[],"[42, 61]",[],"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]",0.0,42.0,0.0,1.0,...,NaN,NaN,NaN,True,NaN,knife,NaN,51.1875,1,Player_4
4,[],[],[],[Player_3],"[0.0, 0.0, 0.0]","[-36.0, 0.0, -27.0]",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,False,NaN,None,NaN,51.1875,1,Player_10


In [ ]:
import os
import json
import pandas as pd
from glob import glob

# Ruta raíz donde clonaste el repositorio
RAIZ = "F:/Tesis/CS2CD.Counter-Strike_2_Cheat_Detection"

# Carpetas de parquets
CARPETA_NO  = os.path.join(RAIZ, "no_cheater_present")
CARPETA_CHE = os.path.join(RAIZ, "with_cheater_present")

# Columnas que queremos conservar de cada Parquet
columnas_relevantes = [
    "X", "Y", "Z", "tick",       
    "steamid",                   
    "velocity_X", "velocity_Y", "velocity_Z",
    "is_airborne", "is_walking",
    "yaw", "pitch",
    "usercmd_mouse_dx", "usercmd_mouse_dy",
    "active_weapon",
    "active_weapon_ammo", "total_ammo_left",
    "shots_fired", "kills_total", "deaths_total",
    "headshot_kills_total",
    "health", "armor_value", "is_alive"
]

# Ruta de salida del CSV final
OUTPUT_CSV = "subset_cs2cd.csv"

# Cuántos shards (archivos .parquet) quieres procesar como muestra
N_SHARDS_NO  = 20  # primeros 20 de no_cheater_present
N_SHARDS_CHE = 20  # primeros 20 de with_cheater_present


# ------------------------------------------------------------------
# 2. FUNCIONES AUXILIARES
# ------------------------------------------------------------------

def listar_shards_ordenados(carpeta, n):
    """
    Devuelve las n primeras rutas .parquet de la carpeta,
    ordenadas por número (0,1,2,...).
    """
    rutas = glob(os.path.join(carpeta, "*.parquet"))
    rutas_ord = sorted(rutas, key=lambda p: int(os.path.basename(p).split(".")[0]))
    return rutas_ord[:n]


def obtener_cheaters_desde_json(ruta_json):
    """
    Lee el JSON y devuelve la lista de steamids que hicieron cheat
    en esa partida (campo "cheaters").
    """
    with open(ruta_json, "r", encoding="utf-8") as f:
        j = json.load(f)
    return [c.get("steamid") for c in j.get("cheaters", [])]


def leer_parquet_filtrado(ruta_parquet):
    """
    Lee un .parquet y devuelve un DataFrame solo con las columnas
    listadas en 'columnas_relevantes' que existan en el archivo.
    """
    df = pd.read_parquet(ruta_parquet)
    cols = [c for c in columnas_relevantes if c in df.columns]
    return df[cols].copy()


# ------------------------------------------------------------------
# 3. PROCESAMIENTO Y ESCRITURA A CSV
# ------------------------------------------------------------------

# Obtener listas de shards ordenados
parquets_no  = listar_shards_ordenados(CARPETA_NO,  N_SHARDS_NO)
parquets_che = listar_shards_ordenados(CARPETA_CHE, N_SHARDS_CHE)

# Borrar CSV previo (si existe)
if os.path.exists(OUTPUT_CSV):
    os.remove(OUTPUT_CSV)

primera_iteracion = True

# procesar shards SIN trampas
for ruta_parq in parquets_no:
    base      = os.path.basename(ruta_parq).replace(".parquet","")
    ruta_json = os.path.join(CARPETA_NO, f"{base}.json")
    cheaters  = obtener_cheaters_desde_json(ruta_json)  # lista vacía

    df_chunk  = leer_parquet_filtrado(ruta_parq)
    df_chunk["match_id"]   = base
    df_chunk["is_cheater"] = df_chunk["steamid"].isin(cheaters).astype(int)

    # Escribir CSV incrementalmente
    if primera_iteracion:
        df_chunk.to_csv(OUTPUT_CSV, index=False, mode="w", header=True)
        primera_iteracion = False
    else:
        df_chunk.to_csv(OUTPUT_CSV, index=False, mode="a", header=False)

    print(f"[no_cheater] {base}.parquet → {len(df_chunk)} filas")

# Procesar shards CON trampas
for ruta_parq in parquets_che:
    base      = os.path.basename(ruta_parq).replace(".parquet","")
    ruta_json = os.path.join(CARPETA_CHE, f"{base}.json")
    cheaters  = obtener_cheaters_desde_json(ruta_json)

    df_chunk  = leer_parquet_filtrado(ruta_parq)
    df_chunk["match_id"]   = base
    df_chunk["is_cheater"] = df_chunk["steamid"].isin(cheaters).astype(int)

    df_chunk.to_csv(OUTPUT_CSV, index=False, mode="a", header=False)
    print(f"[cheater]    {base}.parquet → {len(df_chunk)} filas")

print(f"\nCSV final generado en: {OUTPUT_CSV}")


# ------------------------------------------------------------------
# 4. VERIFICACIÓN FINAL (opcional)
# ------------------------------------------------------------------

df_final = pd.read_csv(OUTPUT_CSV)
print("\nVerificación final:")
print(" - Shape del CSV: ", df_final.shape)
print(" - Columnas:       ", df_final.columns.tolist())
print(" - Etiquetas is_cheater:\n", df_final["is_cheater"].value_counts())

[no_cheater] 0.parquet → 122740 filas
[no_cheater] 1.parquet → 1378120 filas
[no_cheater] 2.parquet → 699680 filas
[no_cheater] 3.parquet → 1164888 filas
[no_cheater] 4.parquet → 1211180 filas
[no_cheater] 5.parquet → 1067050 filas
[no_cheater] 6.parquet → 925440 filas
[no_cheater] 7.parquet → 1946850 filas
[no_cheater] 8.parquet → 97390 filas
[no_cheater] 9.parquet → 1573580 filas
[no_cheater] 10.parquet → 1189160 filas
[no_cheater] 11.parquet → 1360130 filas
[no_cheater] 12.parquet → 1249390 filas
[no_cheater] 13.parquet → 1143440 filas
[no_cheater] 14.parquet → 1003330 filas
[no_cheater] 15.parquet → 1000600 filas
[no_cheater] 16.parquet → 886490 filas
[no_cheater] 17.parquet → 1230680 filas
[no_cheater] 18.parquet → 1096870 filas
[no_cheater] 19.parquet → 568720 filas
[cheater]    0.parquet → 601790 filas
[cheater]    1.parquet → 849500 filas
[cheater]    2.parquet → 370480 filas
[cheater]    3.parquet → 515700 filas
[cheater]    4.parquet → 625840 filas
[cheater]    5.parquet → 59

In [ ]:
from pathlib import Path
import json
import pandas as pd

# ------------------------------------------------------------------
# 1. CONFIGURACIÓN DE RUTAS Y PARÁMETROS
# ------------------------------------------------------------------

# Carpeta raíz donde tienes el repo clonado
BASE = Path("CS2CD.Counter-Strike_2_Cheat_Detection")  # ← ajústala

# Subcarpetas con los shards
NO_DIR  = BASE / "no_cheater_present"
CHE_DIR = BASE / "with_cheater_present"

# Número de shards a procesar de cada carpeta
N_SHARDS_NO  = 1
N_SHARDS_CHE = 2

# Columnas que queremos extraer de cada .parquet
columnas_relevantes = [
    "X","Y","Z","tick","steamid",
    "velocity_X","velocity_Y","velocity_Z",
    "is_airborne","is_walking",
    "yaw","pitch",
    "usercmd_mouse_dx","usercmd_mouse_dy", "usercmd_viewangle_x", "usercmd_viewangle_y", "aim_punch_angle",
    "i_recoil_idx", "fl_recoil_idx",
    "active_weapon","active_weapon_ammo","total_ammo_left",
    "FIRE", "last_shot_time", "next_primary_attack_tick",
    "shots_fired", "kills_total","deaths_total", "assists_total", "damage_total",
    "headshot_kills_total",
    "spotted", "approximate_spotted_by", "is_scoped",
    "health","armor_value","is_alive"
]

# Ruta de salida del CSV
OUTPUT_CSV = Path("subset_test_cs2cd.csv")


# ------------------------------------------------------------------
# 2. FUNCIONES AUXILIARES
# ------------------------------------------------------------------

def listar_shards_ordenados(carpeta: Path, n: int):
    """
    Devuelve las n primeras rutas .parquet de 'carpeta',
    ordenadas numéricamente según el nombre del fichero.
    """
    shards = sorted(carpeta.glob("*.parquet"), key=lambda p: int(p.stem))
    return shards[:n]


def obtener_cheaters_desde_json(ruta_json: Path):
    """
    Lee el JSON y devuelve la lista de steamids que hicieron cheat
    en esa partida (campo "cheaters").
    """
    data = json.loads(ruta_json.read_text(encoding="utf-8"))
    return [c.get("steamid") for c in data.get("cheaters", [])]


def obtener_csstats_info(ruta_json: Path):
    """
    Lee el JSON y extrae el primer dict dentro de "CSstats_info":
    {'map':..., 'server':..., 'avg_rank':..., 'match_making_type':...}
    """
    data = json.loads(ruta_json.read_text(encoding="utf-8"))
    info = data.get("CSstats_info", [])
    return info[0] if info else {}


def leer_parquet_filtrado(ruta_parquet: Path):
    """
    Lee un .parquet y devuelve un DataFrame con solo las columnas
    listadas en 'columnas_relevantes' que existan en ese shard.
    """
    df = pd.read_parquet(ruta_parquet)
    cols = [c for c in columnas_relevantes if c in df.columns]
    return df[cols].copy()


# ------------------------------------------------------------------
# 3. PROCESAMIENTO Y ESCRITURA A CSV
# ------------------------------------------------------------------

# Si existe un CSV previo, lo borramos para partir limpio
if OUTPUT_CSV.exists():
    OUTPUT_CSV.unlink()

primera_iteracion = True

# Función interna para procesar un shard (común a no-cheater y cheater)
def procesar_shard(shard: Path, label: int):
    base     = shard.stem
    ruta_js  = shard.with_suffix(".json")
    # 1) lista de steamids tramposos en esta partida
    cheaters = obtener_cheaters_desde_json(ruta_js)
    # 2) CSstats info de la partida
    csinfo   = obtener_csstats_info(ruta_js)

    # 3) lectura y filtrado
    df = leer_parquet_filtrado(shard)
    # 4) etiqueta por fila
    df["is_cheater"] = df["steamid"].isin(cheaters).astype(int)
    # 5) añadir campos de CSstats_info
    for k, v in csinfo.items():
        df[k] = v

    return df

# 3.1 Procesar shards sin trampas
for shard in listar_shards_ordenados(NO_DIR, N_SHARDS_NO):
    df_chunk = procesar_shard(shard, label=0)
    df_chunk.to_csv(
        OUTPUT_CSV,
        mode="w" if primera_iteracion else "a",
        header=primera_iteracion,
        index=False
    )
    primera_iteracion = False
    print(f"[no_cheater] {shard.name} → {len(df_chunk)} filas")

# 3.2 Procesar shards con trampas
for shard in listar_shards_ordenados(CHE_DIR, N_SHARDS_CHE):
    df_chunk = procesar_shard(shard, label=1)
    df_chunk.to_csv(
        OUTPUT_CSV,
        mode="a",
        header=False,
        index=False
    )
    print(f"[cheater]    {shard.name} → {len(df_chunk)} filas")

print(f"\nCSV final generado en: {OUTPUT_CSV.resolve()}")


# ------------------------------------------------------------------
# 4. VERIFICACIÓN FINAL (OPCIONAL)
# ------------------------------------------------------------------

df_final = pd.read_csv(OUTPUT_CSV)
print("\nVerificación:")
print(" - Shape:", df_final.shape)
print(" - Columnas:", df_final.columns.tolist())
print(" - is_cheater counts:\n", df_final["is_cheater"].value_counts())

[no_cheater] 0.parquet → 122740 filas
[cheater]    0.parquet → 601790 filas
[cheater]    1.parquet → 849500 filas

CSV final generado en: F:\Tesis\subset_test_cs2cd.csv


C:\Users\kilua\AppData\Local\Temp\ipykernel_18608\3873494010.py:141: DtypeWarning: Columns (9,22,31,33,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df_final = pd.read_csv(OUTPUT_CSV)



Verificación:
 - Shape: (1574030, 42)
 - Columnas: ['X', 'Y', 'Z', 'tick', 'steamid', 'velocity_X', 'velocity_Y', 'velocity_Z', 'is_airborne', 'is_walking', 'yaw', 'pitch', 'usercmd_mouse_dx', 'usercmd_mouse_dy', 'usercmd_viewangle_x', 'usercmd_viewangle_y', 'aim_punch_angle', 'i_recoil_idx', 'fl_recoil_idx', 'active_weapon', 'active_weapon_ammo', 'total_ammo_left', 'FIRE', 'last_shot_time', 'next_primary_attack_tick', 'shots_fired', 'kills_total', 'deaths_total', 'assists_total', 'damage_total', 'headshot_kills_total', 'spotted', 'approximate_spotted_by', 'is_scoped', 'health', 'armor_value', 'is_alive', 'is_cheater', 'map', 'server', 'avg_rank', 'match_making_type']
 - is_cheater counts:
 is_cheater
0    1368722
1     205308
Name: count, dtype: int64
